In [62]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
import torch
from dataset import Model10NetDataset
from model import TNet, MLP

In [64]:
torch.cuda.is_available()

True

In [65]:
dataset = Model10NetDataset(path="./ModelNet10")

In [66]:
point_cloud = dataset[0][0]
point_cloud.shape

torch.Size([1601, 3])

In [67]:
point_cloud = point_cloud.to("cuda")
point_cloud.shape

torch.Size([1601, 3])

In [68]:
def stack(x):
    # Add a dimension to make it (1, N, C)
    x_unsqueezed = x.unsqueeze(0)

    # Concatenate along the new dimension to make it (2, N, C)
    x_stacked = torch.cat([x_unsqueezed, x_unsqueezed], dim=0)

    return x_stacked

In [69]:
point_cloud = stack(point_cloud)
point_cloud.shape

torch.Size([2, 1601, 3])

In [73]:
tnet = TNet(in_features=3)
tnet = tnet.to("cuda")
tnet

TNet(
  (linear1): Linear(in_features=3, out_features=64, bias=True)
  (linear2): Linear(in_features=64, out_features=128, bias=True)
  (linear3): Linear(in_features=128, out_features=1024, bias=True)
  (relu): ReLU()
  (batchnorm1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=9, bias=True)
  )
)

In [80]:
out = tnet(point_cloud)
out.shape

torch.Size([2, 1601, 3])


torch.Size([2, 3, 3])

In [53]:
out

tensor([[-0.1096,  0.3557,  0.4177,  0.0255,  0.2147, -0.0453, -0.6353, -0.0045,
          0.4330]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [88]:
mlp = MLP(layers=[3, 64, 128], nonlinear_last=False)
mlp = mlp.to("cuda")
mlp

MLP(
  (layers): ModuleList(
    (0): Linear(in_features=3, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=128, bias=True)
  )
)

In [85]:
mlp(point_cloud)

In [90]:
import torch.nn as nn

In [ ]:
conv = nn.Conv1d(1, 4, 12)